# Merge data with CMS numbers

Data from the following sources:
- 03_teaching_hospital_info.xlsx. EDA link: [01_EDA_teaching_hospital](01_EDA_teaching_hospital.ipynb)
- 04_CMS_hospital_info.csv EDA link: [06_EDA_CMS_hospital_info.ipynb](06_EDA_CMS_hospital_info.ipynb)
- 05_tbl_usr_rpt_5103.xlsx EDA link: [04_EDA_data_from_Sam](04_EDA_data_from_Sam.ipynb)
- 06_tbl_usr_rpt_5104.xlsx EDA link: [04_EDA_data_from_Sam](04_EDA_data_from_Sam.ipynb)

In [3]:
import pandas as pd
import numpy as np

In [4]:
# load data
raw03 = pd.read_excel(
    "../data/hospital_level_info//03_teaching_hospital_info.xlsx",
    index_col=0,
)
raw04 = pd.read_csv("../data/hospital_level_info/04_CMS_hospital_info.csv")
raw05 = pd.read_excel("../data/hospital_level_info/05_tbl_usr_rpt_5103.xlsx")
raw06 = pd.read_excel("../data/hospital_level_info/06_tbl_usr_rpt_5104.xlsx")

In [40]:
# visualize the shape of all the tables
raw03.shape, raw04.shape, raw05.shape, raw06.shape

((1344, 15), (5339, 29), (7024, 20), (7024, 131))

In [43]:
# preprocess add a string ID to raw03 and raw04
raw03['CMS Certification Number'] = raw03['CCN'].astype(str).apply(lambda x: x.zfill(6))
raw04['CMS Certification Number'] = raw04['Facility ID']

In [44]:
# visualize the rows in raw03 that has facility id not in raw04
raw03[~raw03['CMS Certification Number'].isin(raw04['CMS Certification Number'])].head()

,CCN,TIN,Hospital Name,PECOS Legal Business Name,Street Address,PO Box,City,State,Zip Code,Street Address 1,Street Address 2,City.1,State.1,Zip Code.1,Facility ID,CMS Certification Number
ROW,,,,,,,,,,,,,,,,
21,10137,636001579,Cooper Green Mercy Hospital,Jefferson County Commission,1515 Sixth Avenue South,NaN,Birmingham,AL,35233,1515 6th Ave S,NaN,Birmingham,AL,35233,010137,010137
22,10152,203713023,Infirmary West,"Infirmary Health Hospitals, Inc.",5600 Girby Road,144,Mobile,AL,36693,5600 Girby Rd,NaN,Mobile,AL,36693,010152,010152
55,40042,710236932,Crittenden Regional Hospital,Crittenden Hospital Association,200 Tyler St,2248,West Memphis,AR,72303,200 W Tyler Ave,NaN,West Memphis,AR,72301,040042,040042
62,43026,710236856,Baptist Health Rehabilitation Instit,Baptist Health,9601 Baptist Health Drive,NaN,Little Rock,AR,72205,9601 Baptist Health Drive,NaN,Little Rock,AR,72205,043026,043026
96,50146,951683875,City Of Hope National Medical Ctr,City Of Hope National Medical Center,1500 E. Duarte Road,NaN,Duarte,CA,91010,1500 E Duarte Rd,NaN,Duarte,CA,91010,050146,050146


In [45]:
# visualize the rows in raw04 that has facility id not in raw05
raw04[~raw04['CMS Certification Number'].isin(raw05['CMS Certification Number'])].head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,...,Patient experience national comparison,Patient experience national comparison footnote,Effectiveness of care national comparison,Effectiveness of care national comparison footnote,Timeliness of care national comparison,Timeliness of care national comparison footnote,Efficient use of medical imaging national comparison,Efficient use of medical imaging national comparison footnote,Location,CMS Certification Number
46,194085,COMPASS BEHAVIORAL CENTER OF LAFAYETTE,312 YOUNGSVILLE HIGHWAY,LAFAYETTE,LA,70508,LAFAYETTE,(337) 534-4655,Psychiatric,Proprietary,...,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,POINT (-91.991048 30.155424),194085
91,670117,TEXAS GENERAL HOSPITAL- VZRMC LP,707 N WALDRIP ST,GRAND SALINE,TX,75140,VAN ZANDT,(903) 962-5200,Acute Care Hospitals,Proprietary,...,Not Available,16.0,Not Available,5.0,Same as the national average,NaN,Not Available,16.0,POINT (-95.721737 32.679193),670117
107,390302,BARIX CLINICS OF PENNSYLVANIA,280 MIDDLETOWN ROAD,LANGHORNE,PA,19047,BUCKS,(267) 572-3100,Acute Care Hospitals,Proprietary,...,Not Available,16.0,Not Available,5.0,Not Available,16.0,Not Available,16.0,POINT (-74.884542 40.181801),390302
148,244005,CHILD & ADOLESCENT BEHAVIORAL HEALTH SERVICES,1701 TECHNOLOGY DRIVE NE,WILLMAR,MN,56201,KANDIYOHI,(320) 231-5405,Psychiatric,Government - State,...,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,POINT (-95.019258 45.142263),244005
190,194007,NORTHLAKE BEHAVIORAL HEALTH SYSTEM,23515 HIGHWAY 190,MANDEVILLE,LA,70448,ST. TAMMANY,(985) 626-6300,Psychiatric,Voluntary non-profit - Private,...,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,POINT (-90.02169 30.345064),194007


In [72]:
# preprocess merge all the data using CMS ID
merged = raw05.merge(raw06)
merged = merged.merge(raw04, how='outer', on='CMS Certification Number')
merged = merged.merge(raw03, how='outer', on='CMS Certification Number')

In [74]:
merged.to_csv("../intermediate/03_hospital_level_info_merged.csv")

In [81]:
any(["County" in column for column in merged.columns])

True

In [89]:
import joblib
county_fips_dict = joblib.load("county_fips_dict")

In [90]:
county_fips_dict

{'KINGS': '36047',
 'VENTURA': '06111',
 'LOS ANGELES': '06037',
 'RIVERSIDE': '06065',
 'MODOC': '06049',
 'MONTEREY': '06053',
 'ORANGE': '12095',
 'TULARE': '06107',
 'WARE': '13299',
 'CHARLTON': '13049',
 'IBERIA': '22045',
 'TERREBONNE': '22109',
 'RAPIDES': '22079',
 'UNION': '34039',
 'CONCORDIA': '22029',
 'CALDWELL': '48055',
 'EAST BATON ROUGE': '22033',
 'FRANKLIN': '39049',
 'BROWN': '20013',
 'ALAMEDA': '06001',
 'CHATHAM': '13051',
 'FLOYD': '18043',
 'UPSON': '13293',
 'SUMTER': '45085',
 'MERIWETHER': '13199',
 'COWETA': '13077',
 'MONROE': '01099',
 'MILWAUKEE': '55079',
 'FOND DU LAC': '55039',
 'CALCASIEU': '22019',
 'OUACHITA': '22073',
 'JEFFERSON': '01073',
 'ST. JAMES': '22093',
 'ACADIA': '22001',
 'ORLEANS': '22071',
 'LAFOURCHE': '22057',
 'WASHINGTON': '28151',
 'EAST FELICIANA': '22037',
 'EL PASO': '08041',
 'SAN BERNARDINO': '06071',
 'FRESNO': '06019',
 'PROVIDENCE': '44007',
 'SHASTA': '06089',
 'SANTA BARBARA': '06083',
 'STEWART': '13259',
 'SAN DIEGO